<a href="https://colab.research.google.com/github/daikichiba9511/study_log/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
cd /content/drive/My\ Drive/Colab Notebooks/titanic

/content/drive/My Drive/Colab Notebooks/titanic


In [10]:
import pandas as pd 
train=pd.read_csv("./train.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [11]:
#Passenger ID に簡単にアクセスできるようにしておく
PassengerID=test['PassengerID']

KeyError: ignored

In [12]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
#欠損値の確認
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [0]:
#使用するライブラリのインポート
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
 
#しようするモデルのインポート
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
#cross_validationからKFoldはimportできない、model_selectionに入ってる
from sklearn.model_selection import KFold
  
import warnings
warnings.filterwarnings('ignore')
  

In [0]:
#データセットの読み込み
train = pd.read_csv("./train.csv")
test  = pd.read_csv("./train.csv")
  
#Passenger ID に簡単にアクセスできるようにしておく
PassengerID=test['PassengerId']
full_data=[train,test]


In [0]:
#データの整理、整形と新しい特徴量を追加していく
#欠損値も補完する

#乗客の名前の長さを追加
train['Name_length'] = train['Name'].apply(len)
test['Name_length']  = test['Name'].apply(len)

#客室ありの乗客かどうかを追加
train['Has_Canib'] = train['Cabin'].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin']  = test['Cabin'].apply(lambda x:  0 if type(x) == float else 1)

#同乗した兄弟、配偶者の人数(SibSp)と同乗した親。子供の人数(Parch)からファミリーサイズを追加する
for dataset in full_data:
  dataset['FamilySize'] = dataset['SibSp']+dataset['Parch'] + 1
  
#ファミリーサイズから１名の乗客かどうかを追加する
for dataset in full_data:
  dataset['IsAlone']=0
  #datasetの行ラベルを指定、列のfamilysizeが１のとこはisaloneに１を代入
  dataset.loc[dataset['FamilySize']==1, 'IsAlone']=1
  
#乗船した港（Embarked）に欠損値があるので Southmptonで補完する
#C=Cherbourg,Q=Queenstown,S=Southmpton
for dataset in full_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')
  
#運賃（ Fare）に欠損値があるから、中央値で補完
for dataset in full_data:
  dataset['Fare']=dataset['Fare'].fillna(train['Fare'].median())
  
#年齢に欠損値があるから、平均と標準偏差から乱数を生成し、補完する
for dataset in full_data:
  #年齢の平均
  age_ave = dataset['Age'].mean()
  #年齢の標準偏差
  age_std = dataset['Age'].std()
  age_null_count = dataset['Age'].isnull().sum()
  age_null_random_list =np.random.randint(age_ave - age_std, age_ave + age_std, size=age_null_count)
  #datasetのAgeのカラムで欠損値age_nulll_random_listを代入
  dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
  dataset['Age'] = dataset['Age'].astype(int)
  
  
#乗客の名前から敬称（MrやMsなど）を抽出してくる関数を定義する
def get_title(name):
  title_search  = re.search(' ([A-Za-z])\.',name)
  if title_search:
    return title_search.group(1)
  
  return ""

#乗客の名前 (Name)から敬称を抽出し、追加する
for dataset in full_data:
  dataset['Title'] = dataset['Name'].apply(get_title)
  
#敬称(Title)をグルーピングしたもので書き換える
for dataset in full_data:
  dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
  
  dataset['Title'] = dataset['Title'].replace('Mile','Miss')
  dataset['Title'] =dataset ['Title'].replace('Ms','Miss')
  dataset['Title'] = dataset['Title'].replace('Mme','Mrs')
  
#データの整形
for dataset in full_data:
  #性別(Sex)を定数化する
  dataset['Sex'] = dataset['Sex'].map( {'female':0,'male':1}).astype(int)
  
  #敬称(Title)を定数化する
  title_mapping = {"Mr":1,"Miss":2,"Mrs":3,"Master":4,"Rare":5}
  dataset['Title'] = dataset['Title'].map(title_mapping)
  #敬称(Title)がないレコードを０埋め
  dataset['Title'] = dataset['Title'].fillna(0)
  
  #乗船した港(Embarked)を定数化する
  dataset['Embarked'] = dataset['Embarked'].map({'S':0,'C':1,'Q':2})
  
  #年齢を定式化する。
  dataset.loc[ dataset['Age'] <= 16, 'Age']                          = 0
  dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
  dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
  dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
  dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

  
#特徴量の選択（不要なカラムを落とす。）
drop_elements = ['PassengerId','Name','Ticket','Cabin','SibSp']
train = train.drop(drop_elements,axis=1)
test  = test.drop(drop_elements,axis=1)



  
  
  
  
  
  
  

In [0]:
ntrain = train.shape[0]
ntest = test.shape[0]

#交差検証の回数
NFOLDS=5
#同じ乱数を発生させるための固定値
SEED=0

#パラメタの検証、学習モデルの制度を評価
#交差検証（クロスバリデーション）でパラメタを検証し、精度がよく、過学習怒らないパラメタを決定
kf = KFold( n_splits=NFOLDS,random_state=SEED)

#Sklearn classifierを拡張
class SklearnHelper(object):
  def __init__(self,clf,seed=0,params=None):
    params['random_state']=seed
    self.clf = clf(**params)
    
  def train(self,x_train,y_train):
    self.clf.fit(x_train,y_train)
    
  def predict(self,x):
    return self.clf.predict(x)
  
  def fit(self,x,y):
    return self.clf.fit(x,y)
  
  
  
def get_oof(clf,x_train,y_train,x_test):
  oof_train = np.zeros((ntrain,))
  oof_test =np.zeros((ntest,))
  oof_test_skf=np.empty((NFOLDS,ntest))
  
  for i , (train_index,test_index) in enumetate(kf):
    x_tr =x_train[train_index]
    y_tr = y_train[train_index]
    x_te=x_train[test_index]
    
    xlf.train(x_tr,y_tr)
    
    
    oof_train[test_index] = clf.predict(x_te)
    ppf_test_skf[i,:] = clf.predict(x_test)
    
  oof_test[:] =oof_test_skf.mean(axis=0)
  return oof_train.reshape(-1,1),oof_test.reshape(-1,1)


In [0]:
#モデルのパラメタ定義
#random forestのパラメタ
rf_params = {'n_jobs':-1,'n_estimators':500,'warm_start':True,'max_depth':6,'min_samples_liaf':2'max_features':'sqrt':'verbose':0}
#Extra Trees のパラメタ
et_params = {'n_jobs':-1,'n_estimatiors':500}